In [1]:
! pip install langchain openai tiktoken chromadb einops transformers xformers sentencepiece accelerate bitsandbytes docx2txt python-docx sentence_transformers
! pip install pydantic==1.10.11

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 78.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.4/396.4 kB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 113.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 MB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 97.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os, warnings, torch
from transformers import LlamaTokenizer, LlamaForCausalLM, pipeline
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import Docx2txtLoader
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.cache import InMemoryCache
from chromadb.config import Settings

llm_cache = InMemoryCache()

warnings.filterwarnings("ignore")

os.environ["LLM_REPOSITORY"] = "chavinlo/alpaca-native"
os.environ["EMBEDDING_REPOSITORY"] = "sentence-transformers/all-MiniLM-L6-v2"
os.environ["MAX_TOKENS"] = "1024"
os.environ["VECTOR_STORE_FOLDER"] = "vectordb"

In [3]:
tokenizer = LlamaTokenizer.from_pretrained(os.environ["LLM_REPOSITORY"])

alpaca_model = LlamaForCausalLM.from_pretrained(
    os.environ["LLM_REPOSITORY"],
    load_in_8bit=True,
    device_map="auto",
)
# device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
# model.to(device)

pipeline = pipeline(
    "text-generation",
    model=alpaca_model,
    tokenizer=tokenizer,
    max_length=int(os.environ["MAX_TOKENS"]),
    temperature=0.6,
    top_p=0.95,
    repetition_penalty=1.2
)

local_llm = HuggingFacePipeline(pipeline=pipeline)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118_nocublaslt.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118_nocublaslt.so...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
doc_path = "/content/Preliminary_Sample_Summary_For_Few_Shot.docx"
loader = Docx2txtLoader(file_path=doc_path)
doc = loader.load()

In [6]:
prompt_template = """:

"{text}"


SUMMARY:"""
summary_prompt = PromptTemplate(template=prompt_template, input_variables=["text"])


summary_templates = [
    "Provide a summary of the clinical trial for a patient.",
    "You are the PI of the trial. Summarize the trial for a potential patient at a 6th-grade reading level.",
    "You are an oncologist. Describe this trial for your patient at a 6th-grade reading level.",
    "with Chain-of-Thought: Provide a summary of the clinical trial for a patient. Please explain step-by-step."
]

summary_prompts = [PromptTemplate.from_template(f"{template} {prompt_template}") for template in summary_templates]

In [7]:
for i, summary_prompt in enumerate(summary_prompts):
  summary_chain = load_summarize_chain(local_llm, chain_type="map_reduce", combine_prompt=summary_prompt)
  print(f"PROMPT: {summary_templates[i]}\n")
  print(f"SUMARY: {summary_chain.run(doc)}\n\n")

PROMPT: Provide a summary of the clinical trial for a patient.



SUMARY:  
This clinical trial is studying two different treatment approaches for non-luminal A hormonal receptor positive breast cancer - early chemotherapy or endocrine therapy based regimes. The goal is to compare the effectiveness of the treatments on antitumor activity, as well as safety and tolerability. Additionally, the researchers are looking at whether circulating free DNA nucleic acid mutation mutations may be present in participants' bodies at disease progression.


PROMPT: You are the PI of the trial. Summarize the trial for a potential patient at a 6th-grade reading level.

SUMARY:  The INSIGHT study is evaluating two different treatment options for non-luminal A hormone receptorpositive metastatic breast cancer. It will also be looking at the safety and tolerability of each option and detecting any mutations in cfDNA in cfDNA.


PROMPT: You are an oncologist. Describe this trial for your patient at a 6th-grade reading level.

SUMARY:  The INSIGHT study will evaluate the e

In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model_kwargs = {'device': device}
encode_kwargs = {"normalize_embeddings": False}
embedding = HuggingFaceEmbeddings(
    model_name=os.environ["EMBEDDING_REPOSITORY"],
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [9]:
if (
    not os.path.isdir(os.environ["VECTOR_STORE_FOLDER"])
    or
    len(os.listdir(os.environ["VECTOR_STORE_FOLDER"])) == 0):
    docsearch_db = Chroma.from_documents(
        documents=doc,
        persist_directory=os.environ["VECTOR_STORE_FOLDER"],
        embedding=embedding)

    # persist embeddings to ROM
    docsearch_db.persist()
    docsearch_db = None

# Define the Chroma settings
CHROMA_SETTINGS = Settings(
    chroma_db_impl="duckdb+parquet",
    persist_directory=os.environ["VECTOR_STORE_FOLDER"],
    anonymized_telemetry=False,
)

docsearch_db = Chroma(
    embedding_function=embedding,
    client_settings=CHROMA_SETTINGS)

retriever = docsearch_db.as_retriever(search_kwargs={"k":1})

In [10]:
prompt_str = "Answer this question: {question}"
qa_prompt = PromptTemplate.from_template(prompt_str)
qa = RetrievalQA.from_chain_type(
    llm=local_llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True)

In [11]:
queries = [
  "What is the purpose of this study?",
  "What conditions must patients have who are eligible for this trial?",
  "What are the age or gender/sex requirements for this clinical trial?",
  "What drugs will trial subjects be subjected to?",
  "Is this trial still available for enrollment? (basically is current date between start date and end date)",
]

for i, query in enumerate(queries):
    query = qa_prompt.format_prompt(question=query)
    print(f"answer to question {i+1}")
    print(qa(query.text)["result"])
    print("\n")

answer to question 1
 The purpose of this study is to determine the impact of early chemotherapy (capecitabine) versus endocrninether based regimens on antitumor effect inpatients withnon Luminal A hormonononreceptor positivepositive metastatic breast cancer.


answer to question 2
 Patients eligible for this trial must have hist hist have non-Luminal A Luminal A hormore hormore hormore hormore hormore hormore hormore hormore hormore hormore hormore hormore hormore hormore hormore hormore hormore hormore hormore hormore hormore hormore hormore hormore hormore hormore hormore hormore hormore hormore hormore hormore hormore hormore hormore hormore hormore hormore hormore hormore hormore hormore hormore hormore hormore hormore hormore hormore hormore hormore hormore hormore hormore hormore hormore hormore hormore hormore hormore hormore hormore hormore hormore hormore hormore hormore hormore hormore hormore hormore hormore hormore hormore hormore hormore hormore hormore hormore hormore ho